In [1]:
from recogym import Configuration

In [2]:
import gym
from recogym import env_1_args
from recogym.agents.single_action_agent import SingleActionAgent

env_1_args['random_seed'] = 42

env_1_args['num_steps'] = 10

env = gym.make('reco-gym-v1')

env.init_gym(env_1_args)

# Instantiate instance of CFAgent class.

num_products = 1000

num_offline_users = 5
num_online_users = 1000

static_agent = SingleActionAgent(Configuration({
    **env_1_args,
    'num_products': num_products,
}))

# Resets random seed back to 42, or whatever we set it to in env_0_args.

env.reset_random_seed()

num_clicks, num_events = 0, 0

# ****** TRAIN OFFLINE ******

unique_offline_user_id = 0

for _ in range(num_offline_users):

    # Reset env and set done to False.

    unique_offline_user_id += 1

    env.reset()
    done = False
    observation, reward, done = None, 0, False

    while not done: # ----- LOOP

        old_observation = observation

        action, observation, reward, done, info = env.step_offline(observation, reward, done)

        static_agent.train(old_observation, action, reward, done) # TRAIN OFFLINE

# ****** TRAIN ONLINE ******

unique_online_user_id = 0

for _ in range(num_online_users):

    # Reset env and set done to False.

    unique_online_user_id += 1

    env.reset(unique_online_user_id)

    observation, _, done, _ = env.step(None)
    reward = None

    while not done: # ----- LOOP
        
        # print("\n========================== ONLINE LOOP START ===========================")

        # print(f"ONLINE %%%% RUN act() obs sessions {observation.sessions()}")

        action = static_agent.act(observation, reward, done) # create recommendation product id

        # print(f"ONLINE %%%% action {action}, obs sessions {observation.sessions()} --> RUN step()")

        observation, reward, done, info = env.step(action['a'])

        # print(f"ONLINE %%%% calc obs sessions {observation.sessions()}, reward {reward}")

        # Used for calculating click through rate.

        num_clicks += 1 if reward == 1 and reward is not None else 0
        num_events += 1
        
        # print(f"ONLINE %%%% num_clicks: {num_clicks}")
        # print(f"ONLINE %%%% num_events: {num_events}")
        # print(f"ONLINE %%%% CTR {num_clicks / num_events}")
        #
        # print("============================== ONLINE LOOP END ============================\n")

ctr = num_clicks / num_events

# ------------------------------------

print(f"TOTAL num_clicks: {num_clicks}")
print(f"TOTAL num_events: {num_events}")
print(f"TOTAL Click Through Rate: {ctr:.4f}")



SingleActionAgent %%%% num_products: 1000
TOTAL num_clicks: 713
TOTAL num_events: 80353
TOTAL Click Through Rate: 0.0089
